# The Wonderful World of Coffee

This notebook begins to explore the retail sales of a cars dataset scraped by one gracious
 reddit user, We will be predicting Price first, but lets see where the day takes us

The data source is below

https://github.com/jldbc/coffee-quality-database

First things first, lets import our libraries and take a peek at our data.


In [10]:
#Add library references
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


In [22]:
#Upload Data

df_arab = pd.read_csv('https://raw.githubusercontent.com/jldbc/coffee-quality-database/master/data/arabica_data_cleaned.csv') # read in the Arabica data
df_rob = pd.read_csv('https://raw.githubusercontent.com/jldbc/coffee-quality-database/master/data/robusta_data_cleaned.csv') # read in the Robusta data
#Quick Column rename
df_arab.rename(columns={'Unnamed: 0':'Id'}, inplace=True)
df_rob.rename(columns={'Unnamed: 0':'Id'}, inplace=True)
print(df_arab.columns)


Index(['Id', 'Species', 'Owner', 'Country.of.Origin', 'Farm.Name',
       'Lot.Number', 'Mill', 'ICO.Number', 'Company', 'Altitude', 'Region',
       'Producer', 'Number.of.Bags', 'Bag.Weight', 'In.Country.Partner',
       'Harvest.Year', 'Grading.Date', 'Owner.1', 'Variety',
       'Processing.Method', 'Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body',
       'Balance', 'Uniformity', 'Clean.Cup', 'Sweetness', 'Cupper.Points',
       'Total.Cup.Points', 'Moisture', 'Category.One.Defects', 'Quakers',
       'Color', 'Category.Two.Defects', 'Expiration', 'Certification.Body',
       'Certification.Address', 'Certification.Contact', 'unit_of_measurement',
       'altitude_low_meters', 'altitude_high_meters', 'altitude_mean_meters'],
      dtype='object')


Now that we've got the data inputted, let's begin our EDA process by looking at our datatypes and
overall shape of the data.


In [15]:
#Arabica Stats
#print(df_arab.head().append(df_arab.tail()), "\n")
print("Structure of Arabica: ",df_arab.shape,"\n")
print("Summary Statistic's:\n",df_arab.describe(),"\n")
print("Count of missing Arabica values:\n",df_arab.isnull().sum().sort_values(ascending=False),"\n")

#Rob Stats
#print(df_rob.head().append(df_arab.tail()), "\n")
print("Structure of Robusta: ",df_rob.shape,"\n")
print("Summary Statistic's:\n",df_rob.describe(),"\n")
print("Count of missing Robusta values:\n",df_rob.isnull().sum().sort_values(ascending=False),"\n")
#df_arab.describe()
#df_rob.describe()


Structure of Cars:  (1311, 44) 

Summary Statistic's:
         Unnamed: 0  Number.of.Bags        Aroma       Flavor   Aftertaste  \
count  1311.000000     1311.000000  1311.000000  1311.000000  1311.000000   
mean    656.000763      153.887872     7.563806     7.518070     7.397696   
std     378.598733      129.733734     0.378666     0.399979     0.405119   
min       1.000000        0.000000     0.000000     0.000000     0.000000   
25%     328.500000       14.500000     7.420000     7.330000     7.250000   
50%     656.000000      175.000000     7.580000     7.580000     7.420000   
75%     983.500000      275.000000     7.750000     7.750000     7.580000   
max    1312.000000     1062.000000     8.750000     8.830000     8.670000   

           Acidity         Body      Balance   Uniformity   Clean.Cup  \
count  1311.000000  1311.000000  1311.000000  1311.000000  1311.00000   
mean      7.533112     7.517727     7.517506     9.833394     9.83312   
std       0.381599     0.359213 

,Unnamed: 0,Number.of.Bags,Aroma,Flavor,Aftertaste,Acidity,Body,Balance,Uniformity,Clean.Cup,Sweetness,Cupper.Points,Total.Cup.Points,Moisture,Category.One.Defects,Quakers,Category.Two.Defects,altitude_low_meters,altitude_high_meters,altitude_mean_meters
count,1311.000000,1311.000000,1311.000000,1311.000000,1311.000000,1311.000000,1311.000000,1311.000000,1311.000000,1311.00000,1311.000000,1311.000000,1311.000000,1311.000000,1311.000000,1310.000000,1311.000000,1084.000000,1084.000000,1084.000000
mean,656.000763,153.887872,7.563806,7.518070,7.397696,7.533112,7.517727,7.517506,9.833394,9.83312,9.903272,7.497864,82.115927,0.088863,0.426392,0.177099,3.591915,1759.548954,1808.843803,1784.196379
std,378.598733,129.733734,0.378666,0.399979,0.405119,0.381599,0.359213,0.406316,0.559343,0.77135,0.530832,0.474610,3.515761,0.047957,1.832415,0.840583,5.350371,8767.847252,8767.187498,8767.016913
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
25%,328.500000,14.500000,7.420000,7.330000,7.250000,7.330000,7.330000,7.330000,10.000000,10.00000,10.000000,7.250000,81.170000,0.090000,0.000000,0.000000,0.000000,1100.000000,1100.000000,1100.000000
50%,656.000000,175.000000,7.580000,7.580000,7.420000,7.500000,7.500000,7.500000,10.000000,10.00000,10.000000,7.500000,82.500000,0.110000,0.000000,0.000000,2.000000,1310.640000,1350.000000,1310.640000
75%,983.500000,275.000000,7.750000,7.750000,7.580000,7.750000,7.670000,7.750000,10.000000,10.00000,10.000000,7.750000,83.670000,0.120000,0.000000,0.000000,4.000000,1600.000000,1650.000000,1600.000000
max,1312.000000,1062.000000,8.750000,8.830000,8.670000,8.750000,8.580000,8.750000,10.000000,10.00000,10.000000,10.000000,90.580000,0.280000,31.000000,11.000000,55.000000,190164.000000,190164.000000,190164.000000


In [13]:
#lets get a quick look at the arabica data






,Unnamed: 0,Number.of.Bags,Harvest.Year,Fragrance...Aroma,Flavor,Aftertaste,Salt...Acid,Bitter...Sweet,Mouthfeel,Uniform.Cup,...,Balance,Cupper.Points,Total.Cup.Points,Moisture,Category.One.Defects,Quakers,Category.Two.Defects,altitude_low_meters,altitude_high_meters,altitude_mean_meters
count,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,...,28.000000,28.000000,28.000000,28.000000,28.000000,28.0,28.000000,25.00000,25.000000,25.000000
mean,14.500000,168.000000,2013.964286,7.702500,7.630714,7.559643,7.657143,7.675714,7.506786,9.904286,...,7.541786,7.761429,80.868929,0.065714,2.964286,0.0,1.892857,1367.60000,1387.600000,1377.600000
std,8.225975,143.226317,1.346660,0.296156,0.303656,0.342469,0.261773,0.317063,0.725152,0.238753,...,0.526076,0.330507,2.441233,0.058464,12.357280,0.0,2.601129,838.06205,831.884207,833.980216
min,1.000000,1.000000,2012.000000,6.750000,6.670000,6.500000,6.830000,6.670000,5.080000,9.330000,...,5.250000,6.920000,73.750000,0.000000,0.000000,0.0,0.000000,40.00000,40.000000,40.000000
25%,7.750000,1.000000,2013.000000,7.580000,7.560000,7.397500,7.560000,7.580000,7.500000,10.000000,...,7.500000,7.580000,80.170000,0.000000,0.000000,0.0,0.000000,795.00000,795.000000,795.000000
50%,14.500000,170.000000,2014.000000,7.670000,7.710000,7.670000,7.710000,7.750000,7.670000,10.000000,...,7.670000,7.830000,81.500000,0.100000,0.000000,0.0,1.000000,1095.00000,1200.000000,1100.000000
75%,21.250000,320.000000,2015.000000,7.920000,7.830000,7.770000,7.830000,7.830000,7.830000,10.000000,...,7.830000,7.920000,82.520000,0.120000,0.000000,0.0,2.000000,1488.00000,1488.000000,1488.000000
max,28.000000,320.000000,2017.000000,8.330000,8.080000,7.920000,8.000000,8.420000,8.250000,10.000000,...,8.000000,8.580000,83.750000,0.130000,63.000000,0.0,9.000000,3170.00000,3170.000000,3170.000000
